In [1]:
import ase.db
from ase import atoms
from ase.io import read
import numpy as np
from ase import Atom,Atoms
from ase.neighborlist import NeighborList
from mendeleev import *

In [2]:
db = ase.db.connect('../../Database/g6/ptag.db')

In [3]:
# extract neighborlist based on pure.traj
atoms = read('pure.traj')
radius = [1.5]*64 # set 1.5 to make sure surf atoms have 10 neighbors and bulk atoms have 14 neighbors.  
nl = NeighborList(radius, self_interaction=False, bothways=True)
nl.update(atoms)
# neighbor store the neighbor atoms index in PdAu.
neighbor_list = {}
coordination_list = {}
adsorption_list = {}
for index in range(len(atoms)):
    indices, offsets = nl.get_neighbors(index)
    indices = indices.tolist()
    neighbor_list[index] = indices 
    coordination_list[index] = len(indices)

In [4]:
# extract adsorptionlist based on pure.traj
# adsorption_list保存每一个位点对应的三层shell
adsorption_coordination_list = {} #按照配位数划分

# 读取pure.traj，扩展3倍
atoms = read('pure.traj') 
extend_atoms = atoms*[3,3,1]

# fcc_indices记录扩胞后16个组成fcc位点的3个原子标号
fcc_indices = [[268,270,269],[270,300,271],[300,302,301],[302,460,303],
              [269,271,284],[271,301,286],[301,303,316],[303,461,318],
              [284,286,285],[286,316,287],[316,318,317],[318,319,476],
              [285,287,332],[287,317,334],[317,319,364],[319,477,366]]

# 通过近邻判断原子数
radius = [1.5]*576 # set 1.5 to make sure surf atoms have 10 neighbors and bulk atoms have 14 neighbors.  
nl = NeighborList(radius, self_interaction=False, bothways=True)
nl.update(extend_atoms)
neighbor_extend_list = {}
coordination_extend_list ={}
for index in range(len(extend_atoms)):
    indices, offsets = nl.get_neighbors(index)
    indices = indices.tolist()
    neighbor_extend_list[index] = indices
    coordination_extend_list[index] = len(indices)

# 每一个fcc位点划分6层特征，6层原子数分别为3，3，3，6，3，40
for num in range(16):
    first_shell = fcc_indices[num]
    
    second_shell = []
    for i in first_shell:
        neighbor_i = neighbor_extend_list[i]
        for j in neighbor_i:
            if j not in first_shell and j not in second_shell:
                second_shell.append(j)
    
    third_shell = []
    for i in second_shell:
        neighbor_i = neighbor_extend_list[i]
        for j in neighbor_i:
            if j not in second_shell and j not in third_shell:
                third_shell.append(j)
    
    # divide second shell into three sub shells by distance
    position1 = extend_atoms[first_shell[0]].position+[0,0,1.2]
    position2 = extend_atoms[first_shell[1]].position+[0,0,1.2]
    position3 = extend_atoms[first_shell[2]].position+[0,0,1.2]
    center_position = (position1+position2+position3)/3
    second_shell_distance = {}
    for index in second_shell:
        second_shell_distance[index] = np.linalg.norm(center_position-extend_atoms[index].position)
    second_shell_distance_ordered = sorted(second_shell_distance.items(), key=lambda item:item[1])
    second_shell_distance_sub1 = [second_shell_distance_ordered[i][0] for i in range(0,3)] 
    second_shell_distance_sub2 = [second_shell_distance_ordered[i][0] for i in range(3,6)]
    second_shell_distance_sub3 = [second_shell_distance_ordered[i][0] for i in range(6,12)]
    second_shell_distance_sub4 = [second_shell_distance_ordered[i][0] for i in range(12,15)]
    
    adsorption_shell = []
    adsorption_shell.append([i%64 for i in first_shell])
    adsorption_shell.append([i%64 for i in second_shell_distance_sub1])
    adsorption_shell.append([i%64 for i in second_shell_distance_sub2])
    adsorption_shell.append([i%64 for i in second_shell_distance_sub3])
    adsorption_shell.append([i%64 for i in second_shell_distance_sub4])
    adsorption_coordination_list[num] = adsorption_shell

neighbor_multi_list = []
for i in range(5*64,6*64):
    neigh_i_dict = {}
    first_shell_i = neighbor_extend_list[i].copy()
    total_shell_i = first_shell_i.copy()
    
    second_shell_i = []
    for j in first_shell_i:
        shell_ij = neighbor_extend_list[j].copy()
        for k in shell_ij:
            if (k not in total_shell_i) and (k not in second_shell_i):
                second_shell_i.append(k)
                total_shell_i.append(k)
            elif k not in total_shell_i:
                total_shell_i.append(k)
    
    third_shell_i = []
    for j in second_shell_i:
        shell_ij = neighbor_extend_list[j].copy()
        for k in shell_ij:
            if (k not in total_shell_i) and (k not in third_shell_i):
                third_shell_i.append(k)
                total_shell_i.append(k)
            elif k not in total_shell_i:
                total_shell_i.append(k)

    fourth_shell_i = []
    for j in third_shell_i:
        shell_ij = neighbor_extend_list[j].copy()
        for k in shell_ij:
            if (k not in total_shell_i) and (k not in fourth_shell_i):
                fourth_shell_i.append(k)
                total_shell_i.append(k)
            elif k not in total_shell_i:
                total_shell_i.append(k)
    
    fifth_shell_i = []
    for j in fourth_shell_i:
        shell_ij = neighbor_extend_list[j].copy()
        for k in shell_ij:
            if (k not in total_shell_i) and (k not in fifth_shell_i):
                fifth_shell_i.append(k)
                total_shell_i.append(k)
            elif k not in total_shell_i:
                total_shell_i.append(k)
    
    neigh_i_dict[1] = [i%64 for i in first_shell_i.copy()]
    #print('first shell:',len(first_shell_i))
    neigh_i_dict[2] = [i%64 for i in second_shell_i.copy()]
    #print('second shell:',len(second_shell_i))
    neigh_i_dict[3] = [i%64 for i in third_shell_i.copy()]
    #print('third shell:',len(third_shell_i))
    neigh_i_dict[4] = [i%64 for i in fourth_shell_i.copy()]
    #print('fourth shell:',len(fourth_shell_i))
    neigh_i_dict[5] = [i%64 for i in fifth_shell_i.copy()]
    neighbor_multi_list.append(neigh_i_dict)

In [5]:
def add_to_list(list_target, shell_pd, shell_coord_pd, shell_au, shell_coord_au):
    if len(shell_pd) !=0:
        list_target.append(len(shell_pd))
        list_target.append(np.mean(shell_coord_pd))
    else:
        list_target.append(0)
        list_target.append(0)

    if len(shell_au) !=0:
        list_target.append(len(shell_au))   # number  
        list_target.append(np.mean(shell_coord_au)) # mean of coordination number 

    else:
        list_target.append(0)
        list_target.append(0)
        
    return list_target

## extract ads feature from ptni.db

In [6]:
Adsorption_coordination_feature = []
Ratio = []
Ads_energy = []
Feature = []
Index = []
Sro = []
Label = []
Activity= []


for row in db.select(jobtype='ads',status='relaxed'):
    print(row.id)
    sid = row.sid
    begin_index = sid.find('0x')
    config_code = sid[begin_index:-3]
    config_str = '{:064b}'.format(int(config_code,16))
    config_int = [int(i) for i in config_str]
    au_atom_list = [i for i,x in enumerate(config_int) if x==1]
   
    first_shell = [int(row.uid[-3:]),int(row.uid[-6:-3]),int(row.uid[-9:-6])]
    for i in range(16):
        if sorted(adsorption_coordination_list[i][0]) == sorted(first_shell):
            fcc_index = i
            break
    
    adsorption_coordination_feature = []
    for j in range(5):
        shell = adsorption_coordination_list[fcc_index][j]
        shell_pd = [k for k in shell if k not in au_atom_list]
        shell_pd_coord = [coordination_list[k] for k in shell_pd]
        shell_au = [k for k in shell if k in au_atom_list]
        shell_au_coord = [coordination_list[k] for k in shell_au]
        adsorption_coordination_feature = add_to_list(adsorption_coordination_feature, shell_pd, shell_pd_coord, shell_au, shell_au_coord)
            
    atoms = db.get_atoms(id=row.id)
    ratio = len(au_atom_list)/64
    sur_energy = db.get(sid=row.sid,jobtype='sur').energy
    ads_energy = row.energy
    eo = ads_energy-sur_energy+7.46
    if eo < 1.58:
        activity = 0.600*(eo+0.23)-1.376
    else:
        activity = -1.645*(eo+0.23)+2.688
    
    Activity.append(activity)
    Adsorption_coordination_feature.append(adsorption_coordination_feature)
    Ratio.append(ratio)

    feature =adsorption_coordination_feature + [ratio]
    Feature.append(feature)
    Index.append(row.id)
    Ads_energy.append(eo)
    
np.savetxt('ratio.csv',Ratio,delimiter=',')
np.savetxt('activity.csv',Activity,delimiter=',')
np.savetxt('energy.csv',Ads_energy,delimiter=',')
np.savetxt('ptag_feature.csv',Feature,delimiter=',')

2
4
5
6
8
9
10
12
13
14
16
17
18
20
21
22
24
25
26
28
29
30
32
34
35
36
38
39
40
42
43
44
46
47
48
50
51
52
54
55
57
58
59
61
62
63
65
66
67
69
70
71
73
74
75
77
78
79
81
82
83
85
86
87
89
90
91
93
94
95
97
98
99
101
102
103
106
107
109
110
111
113
114
115
117
118
119
121
122
123
125
126
127
129
130
131
133
134
135
137
138
139
141
142
143
146
147
149
150
151
153
154
155
157
158
159
161
162
163
165
166
167
169
170
171
173
174
175
177
178
179
181
182
183
185
186
187
189
190
191
193
194
195
197
198
199
201
202
203
205
206
207
209
210
211
213
214
215
217
218
219


## extract all sites feature from ptco.db

In [7]:
Global_feature = []
Adsorption_feature = []
Ratio = []
Strain = []
Ads_energy = []
Feature = []
Index = []
Bop = []
Label = []
Activity= []

for row in db.select(jobtype='sur',status='relaxed'):
    print(row.id)
    sid = row.sid
    begin_index = sid.find('0x')
    config_code = sid[begin_index:-3]
    config_str = '{:064b}'.format(int(config_code,16))
    config_int = [int(i) for i in config_str]
    au_atom_list = [i for i,x in enumerate(config_int) if x==1]
    
    ratio = len(au_atom_list)/64
    for i in range(16):
        adsorption_coordination_feature = []
        for j in range(5):
            shell = adsorption_coordination_list[i][j]
            shell_pd = [k for k in shell if k not in au_atom_list]
            shell_pd_coord = [coordination_list[k] for k in shell_pd]
            shell_au = [k for k in shell if k in au_atom_list]
            shell_au_coord = [coordination_list[k] for k in shell_au]
            adsorption_coordination_feature = add_to_list(adsorption_coordination_feature, shell_pd, shell_pd_coord, shell_au, shell_au_coord)
        feature = adsorption_coordination_feature + [ratio]
        Ratio.append(ratio)
        Feature.append(feature)
np.savetxt('ratio_all_sites.csv',Ratio,delimiter=',')
np.savetxt('ptag_feature_all_sites.csv',Feature,delimiter=',')

1
3
7
11
15
19
23
27
31
33
37
41
45
49
53
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
